# EDA — Analytics Table (Day 4)


## Setup + imports


In [17]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

DATA = Path("../data/processed/orders_analytics.parquet")
FIGS = Path("../reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)


## Load processed data


In [18]:

df = pd.read_parquet(DATA)


print("rows:", len(df), "cols:", len(df.columns))


print("\nDtypes (first 15):")
print(df.dtypes.head(15))

missing = df.isna().sum().sort_values(ascending=False).head(10)
print("\nTop missing columns:")
print(missing)

rows: 5 cols: 17

Dtypes (first 15):
order_id                        object
user_id                         object
amount                         float64
quantity                       float64
created_at         datetime64[ns, UTC]
status                          object
status_clean            string[python]
amount__isna                      bool
quantity__isna                    bool
date                            object
year                           float64
month                   string[python]
dow                             object
hour                           float64
amount__outlier                   bool
dtype: object

Top missing columns:
amount            1
dow               1
quantity          1
created_at        1
hour              1
year              1
month             1
date              1
order_id          0
quantity__isna    0
dtype: int64


## Quick audit


In [ ]:
orders_by_hour = (
    df.groupby("hour", dropna=False)
      .agg(n_orders=("order_id", "count"))
      .reset_index()
      .sort_values("hour")
)

orders_by_hour
fig = px.line(orders_by_hour, x="hour", y="n_orders", markers=True,title="Number of orders by hour of day"
)

fig.update_layout(title={"x": 0.05})
fig.update_xaxes(title_text="Hour of day")
fig.update_yaxes(title_text="Number of orders")

save_fig(fig, FIGS / "orders_by_hour.png")
fig


In [ ]:
rev = (
    df.groupby("country", dropna=False)
      .agg(n=("order_id", "size"), revenue=("amount", "sum"),aov=("amount", "mean"),
      )
      .reset_index()
      .sort_values("revenue", ascending=False)
)

rev
fig = px.bar(rev, x="country", y="revenue", title="Revenue by country (all data)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
save_fig(fig, FIGS / "revenue_by_country.png")
fig



## Questions + results


In [20]:
refund_by_country = (
    df.assign(is_refund=df["status_clean"].eq("refund"))
      .groupby("country", dropna=False)
      .agg(
          n_orders=("order_id", "count"),
          n_refunds=("is_refund", "sum"),
      )
      .reset_index()
)

refund_by_country["refund_rate"] = (
    refund_by_country["n_refunds"] / refund_by_country["n_orders"]
)

refund_by_country = refund_by_country.sort_values("refund_rate", ascending=False)

refund_by_country
fig = px.bar(
    refund_by_country,
    x="country",
    y="refund_rate",
    title="Refund rate by country",
    text=refund_by_country["refund_rate"].map(lambda x: f"{x:.1%}")
)

fig.update_layout(title={"x": 0.05})
fig.update_yaxes(tickformat=".0%")
save_fig(fig, FIGS / "refund_rate_by_country.png")
fig


In [11]:
trend = (
    df.groupby("month", dropna=False)
      .agg(n=("order_id","size"), revenue=("amount","sum"))
      .reset_index()
      .sort_values("month")
)

trend
fig = px.line(trend, x="month", y="revenue", title="Revenue over time (monthly)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue")
save_fig(fig, FIGS / "revenue_trend_monthly.png")
fig


## Bootstrap comparison


In [22]:
fig = px.box(df,x="status_clean",y="amount",title="Order amount distribution by status",points="all")

fig.update_layout(title={"x": 0.05})
fig.update_xaxes(title_text="Order status")
fig.update_yaxes(title_text="Order amount")

save_fig(fig, FIGS / "amount_by_status_box.png")
fig



In [ ]:
lo = df["amount"].quantile(0.05)
hi = df["amount"].quantile(0.95)
df["amount_winsor"] = df["amount"].clip(lower=lo, upper=hi)
fig = px.histogram(df, x="amount_winsor", nbins=30,
title="Order amount distribution (winsorized)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
save_fig(fig, FIGS / "amount_hist_winsor.png")
fig


## Findings + caveats


In [13]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))
res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
res


n_SA: 4 n_AE: 1


{'diff_mean': -1.0, 'ci_low': -1.0, 'ci_high': -1.0}